<a href="https://colab.research.google.com/github/Nat-Nimit/programming/blob/master/PA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [44]:
cd drive/'My Drive'/PA/nlp/nlp

[Errno 2] No such file or directory: 'drive/My Drive/PA/nlp/nlp'
/content/drive/My Drive/PA/nlp/nlp


In [3]:
!pip install pythainlp

    100% |████████████████████████████████| 10.3MB 3.0MB/s 
    100% |████████████████████████████████| 276kB 24.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built marisa-trie


In [0]:
from pythainlp import word_tokenize
data = open("title_classification.train")
title = []
label = []
for line in data.readlines():
  data_list = line.strip("\n").split("\t",1)
  label.append(data_list[0])
  title.append(data_list[1])

In [0]:
def feature_matrices(token_list):
  feature_matrix = []
  for i in range(0,len(token_list)):
    vocab = {token:1 for token in token_list[i]}
    feature_matrix.append(vocab)
  return feature_matrix

def clean_data(token_list):
  clean_list = []
  erase_list = ["\'","\"", ",",":"]
  for i in range(0,len(token_list)):
    cleaned = " "
    for alpha in token_list:
      if alpha in erase_list:
        cleaned = cleaned + token_list[i].strip(alpha)
        clean_list.append(cleaned)
  return clean_list

In [53]:

feature_matrix_train = feature_matrices(title)
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=True)
sparse_feature_matrix_train = dv.fit_transform(feature_matrix_train)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(sparse_feature_matrix_train, label)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
from pythainlp import word_tokenize
import joblib
class MaxEntTopicClassifier:

    def load_model(self, model_file_name:str):
      mapping, parameter = joblib.load(model_file_name)
      self.modelmaxent = parameter
      self.mapping = mapping

    def classify(self, title_text:str) -> str: 
      text = {token : 1 for token in word_tokenize(title_text)}
      answer = self.modelmaxent.predict(self.mapping.transform(text))
      return answer

    def classify_list(self, title_text_list:list) -> list:
        """Classify a list of titles

        Some models are faster to classify a list of titles than for-loop over each one.
        You should override this method. 
        """
        answer_list = [self.classify(title) for title in title_text_list ]
        return answer_list

      
import random
class RandomTopicClassifier:

    def load_model(self, model_file_name:str):
        """Load the parameters (model) from files

        The parameters for this classifier do not do anything useful.
        This is just an example
        """
        mapping, parameter = joblib.load(model_file_name)
        self.mapping = mapping
        self.parameter = parameter
        random.seed(parameter)
        

    def classify(self, title_text:str) -> str:
        """Classify randomly

        This classifier returns a random label string
        """
        num_classes = len(self.parameter.classes_)
        classification = random.randint(0, num_classes - 1)
        return self.parameter.classes_[classification]
    
    def classify_list(self, title_list_names):
      answer = [self.classify(title) for title in title_list_names]
      return answer

class DANTopicClassifier:

    def load_model(self, model_file_name:str):
        pass

    def classify(self, title_text:str) -> str: 
        return 'การเมือง'



In [46]:
"""Run all of the classifiers on training and dev data

To make sure that their accuracies look OK.
You must not change anything in this file.
"""
from sklearn.metrics import accuracy_score, classification_report

from topic_classifier import TopicClassifier, MajorityTopicClassifier
#import RandomTopicClassifier
#from maxent_topic_classifier  import MaxEntTopicClassifier
from dan_topic_classifier import DANTopicClassifier
from dan_topic_classifier2 import DANTopicClassifier2

def read_data(data_file_name:str):
    labels = []
    titles = []
    with open(data_file_name) as f:
        for line in f:
            label, title = line.split('\t', 1)
            labels.append(label)
            titles.append(title)
    return (labels, titles)       

def evaluate_classifier(classifier:TopicClassifier, data_file_name:str):
    true_labels, titles = read_data(data_file_name)
    prediction = classifier.classify_list(titles)
    #print (classification_report(true_labels, prediction))
    print (accuracy_score(true_labels, prediction))

if __name__ == '__main__':

    rtc = RandomTopicClassifier()
    rtc.load_model('model.bin')
    print(rtc.classify('นายกลุยภาคใต้'))
    print('Random classification train and test accuracies')
    evaluate_classifier(rtc, 'title_classification.train')
    evaluate_classifier(rtc, 'title_classification.dev')

    '''mjc = MajorityTopicClassifier()
    print('Majority classification train and test accuracies')
    evaluate_classifier(mjc, 'title_classification.train')
    evaluate_classifier(mjc, 'title_classification.dev')'''
 
    maxent = MaxEntTopicClassifier()
    maxent.load_model('model.bin')
    print('MaxEnt model train and test accuracies')
    evaluate_classifier(maxent, 'title_classification.train')
    evaluate_classifier(maxent, 'title_classification.dev')

    '''dan1 = DANTopicClassifier('dan_model.bin')
    print('DAN model train and test accuracies')
    evaluate_classifier(dan1, 'title_classification.train')
    evaluate_classifier(dan1, 'title_classification.dev')

    dan2 = DANTopicClassifier2('dan2_model.bin')
    print ('DAN model2 train and test accuracies')
    evaluate_classifier(dan2, 'title_classification.train')
    evaluate_classifier(dan2, 'title_classification.dev')'''

สิ่งแวดล้อม
Random classification train and test accuracies
0.08268124407609055
0.08656071593889832
MaxEnt model train and test accuracies
0.8585535741838781
0.6315383428483259


In [49]:
!wget http://www.arts.chula.ac.th/ling/wp-content/uploads/TNCc5model.bin  

--2019-03-02 02:57:25--  http://www.arts.chula.ac.th/ling/wp-content/uploads/TNCc5model.bin
Resolving www.arts.chula.ac.th (www.arts.chula.ac.th)... 161.200.48.9
Connecting to www.arts.chula.ac.th (www.arts.chula.ac.th)|161.200.48.9|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39383847 (38M) [application/octet-stream]
Saving to: ‘TNCc5model.bin.1’

TNCc5model.bin.1    100%[===================>]  37.56M  7.25MB/s    in 5.4s    

2019-03-02 02:57:31 (6.91 MB/s) - ‘TNCc5model.bin.1’ saved [39383847/39383847]



In [51]:
!pip install tltk

    100% |████████████████████████████████| 10.5MB 3.0MB/s 
    100% |████████████████████████████████| 757kB 16.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/12/7b/32/5dccd202a0896e39b2adf82252e7e508560ce64e0a6bdbec6a
Successfully built tltk


In [170]:
from pythainlp import word_tokenize
import tltk
def clean_data(title):
  tk = word_tokenize(title)
  erase_list = ["\'","\"", ",",":"," ","\s", "-"]
  for i in range(0, len(tk)):
    if tk[i] != "":
      for e in erase_list:
         y = tk[i].strip(e)
         tk[i] = y
  return tk

tt = clean_data(title[10])

def title_to_w2v(title_tokens):
  tltk.corpus.w2v_load()
  vect = [tltk.corpus.w2v(t)for t in tk]
  new_vect = [ v  for v in vect if v != ()]
  total = len(new_vect)
  score = sum(new_vect)/total
  #print(title_tokens, len(vect), total)
  return score

title_to_w2v(tt)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


array([ 0.66975874, -0.07039134, -0.6568924 , -0.35951537,  0.807786  ,
       -0.78304064, -0.363347  ,  0.06952314, -0.5563734 ,  0.3152643 ,
       -0.32270417,  0.3132618 ,  0.14647278, -0.0722758 ,  0.1897187 ,
       -0.63453496, -1.0876755 ,  0.55601585, -1.5317719 ,  0.73401904,
       -0.20480767, -0.27491006,  1.1255376 , -0.61270654, -0.8677467 ,
       -1.1733923 ,  0.73311245,  0.24002978, -0.5794016 ,  0.95699805,
        0.5726679 ,  0.12154492, -0.01697617, -0.14300957, -0.1332423 ,
        0.45896286, -0.40979868, -0.49533957,  0.27328616, -0.70939416,
        0.11675704, -0.32048202,  0.01758637,  0.7025831 , -0.12782298,
       -0.43072253,  0.21401462, -0.60836124,  0.30287063,  0.33807027,
       -0.4808868 ,  0.4368609 ,  0.16393843,  0.22849007, -0.19197884,
        0.33541617,  0.3229007 ,  0.9921147 ,  0.11113813, -1.0551497 ,
        0.2308183 ,  0.23236394, -1.1600754 , -0.5617396 ,  0.59898245,
        0.34760067, -1.3854996 ,  0.9512762 , -0.9568508 , -0.42

In [0]:
import numpy as np
embedding_size = 100
num_data = len(title)
feature_matrix = np.zeros((num_data, embedding_size))
for num in range(0,num_data):
  tokens = clean_data(title[num])
  feature_matrix[num] = title_to_w2v(tokens)
